In [6]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

# !curl -L -o ./data/car-parts-and-car-damages.zip https://www.kaggle.com/api/v1/datasets/download/humansintheloop/car-parts-and-car-damages


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2959M  100 2959M    0     0  17.1M      0  0:02:52  0:02:52 --:--:-- 18.2M


In [11]:
!pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"

     |████████████████████████████████| 52 kB 815 kB/s eta 0:00:011
     |████████████████████████████████| 241 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 259 kB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 77 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 953 kB 27.6 MB/s eta 0:00:01
     |████████████████████████████████| 506 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 26.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 6.2 MB/s  eta 0:00:01
  Created wheel for oss2: filename=oss2-2.17.0-py3-none-any.whl size=112375 sha256=eae052e9dedd82b9df20caa43ff9addce43aec86d39f7dc8c2ba0b401b22b23b
  Stored in directory: /home/ak

In [12]:
!pip install mmsegmentation

     |████████████████████████████████| 2.4 MB 3.3 MB/s eta 0:00:01


In [13]:
# Installing Segmentation Models for SegFormer, Loss functions and Metrics
# !pip install -U segmentation-models-pytorch

In [8]:
# !pip install -U skimage

In [1]:
datadir = "./data/car-parts-and-car-damages/"

## Intializing Libraries

In [5]:
import zipfile
import random
import os
import json
import numpy as np
import cv2
from sklearn.metrics import f1_score, roc_auc_score
import torchvision
import torch
from torchvision.io import read_image
import matplotlib.pyplot as plt
from PIL import Image,ImageColor
from tqdm import tqdm
from torch.utils.data import DataLoader,Dataset
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
# from torchvision import tv_tensors
# from torchvision.transforms.v2 import functional as F
# from skimage.draw import polygon
from torchvision import transforms
from torch import nn
from segmentation_models_pytorch import MAnet
from  segmentation_models_pytorch.losses import DiceLoss,JaccardLoss
from segmentation_models_pytorch.metrics import get_stats,iou_score,f1_score

In [3]:
# Declaring folder paths for images and masks
carparts_dir = datadir+"/Car_parts_dataset/"
cardamages_dir = datadir+"/Car_damages_dataset/"
carparts_imgs = carparts_dir+"/File1/img/"
carparts_anns = carparts_dir+"/File1/ann/"
cardamages_imgs = cardamages_dir+"/File1/img/"
cardamages_anns = cardamages_dir+"/File1/ann/"
# sizes=(320,320)

## Methods and Classes Initializations

In [4]:
# Retrieving Classes and Ids from Meta Data
def retrieve_meta_data(path=carparts_dir+"/meta.json"):

    MetaJson = json.load(open(path,"r"))
    classes_carprts = []
    associated_colors_carprts = []
    for cls in MetaJson["classes"]:
        classes_carprts.append(cls["title"])
        associated_colors_carprts.append(cls['id'])
    return classes_carprts,associated_colors_carprts

In [9]:
# Dataset Class for Loading Car Parts or Car Damages Datasets
class CarDataLoader(Dataset):
    def __init__(self,transforms,imgs_path,annotations_path,classes,sizes):
        """
        Initialize Dataset class with Car Parts or Car Damages Datasets
        transforms -> Transformations
        imgs_path -> Path for Images
        annotaions_path -> Path for Annotations
        classes - > List of Classes
        sizes -> Tuple of Height and Width

        """
        self.imgs_path =imgs_path
        self.annotations_path = annotations_path
        self.classes = classes
        self.images = list(sorted(os.listdir(imgs_path)))
        self.annotations = [x+".json" for x in self.images]
        self.transforms = transforms
        self.sizes = sizes

    def getMask(sizes,annfile,classes):
        """
        Get Masks After Processing and Stacking multiple label channels
        sizes -> Tuples of Height and Width
        annfile -> Annotations file of image
        classes -> List of Class ids
        """
        img_height,img_width = annfile["size"]["height"],annfile["size"]["width"]
        mask = torch.zeros((img_height,img_width),dtype=torch.long)
        mask_numpy = mask.numpy()
        class_ids =[]
        for object_ in annfile["objects"]:
            class_id = classes.index(object_["classId"])
            class_ids.append(class_id)
            polygon_ = []
            for i in object_["points"]["exterior"]:
                polygon_.append(i)
            polygon_ = np.asarray(polygon_)
            x_,y_ = polygon(polygon_[:,0],polygon_[:,1],(annfile["size"]["width"],annfile["size"]["height"]))
            mask_numpy[y_,x_] = class_id+1

        mask_tensor = transforms.Resize(sizes)(torch.from_numpy(mask_numpy).unsqueeze(0)).squeeze(1)
        mask_numpy = mask_tensor.numpy()
        masks = [(mask_numpy  == class_id) for class_id in range(len(classes)+1)]
        masks_numpy = np.stack(masks,axis=-1).astype('int')
        return masks_numpy


    def __getitem__(self,idx):
        """
        Iterate through the Dataset

        idx -> Index of the Data
        """
        img_path = os.path.join(self.imgs_path, self.images[idx])
        ann_path = os.path.join(self.annotations_path, self.annotations[idx])
        img = Image.open(img_path)
        ann = json.load(open(ann_path,"r"))

        annotated_mask = CarDataLoader.getMask(self.sizes,ann,self.classes)
        img_tensor = self.transforms(img)[:3,:,:]
        mask_tensor = torch.from_numpy(annotated_mask).permute(0,-1,1,2)

        return img_tensor,mask_tensor
    def __len__(self):
        """
        Invoked on len() method returns length
        """
        return len(self.images)

In [ ]:
def TrainandValidate(model,optim,criterion,dataloader,learning_rate,epochs,checkpointName,classes,checkpoint=True):
    """
    For training and Validating Segmentation models
    model [nn.Module]  ->   Model to Train
    optim     ->     Optimizer
    criterion -> Loss Function
    dataloader -> Batch loader object for Dataset
    learning_rate -> Learning rate parameter for model training
    epochs -> Number of epochs or Iterations
    Checkpoint Name -> Save Model Checkpoints at an Interval with Name
    Classes -> List of Class Ids
    """
    for epoch in range(epochs):
        # Initiate TQDM for Progress
        loop = tqdm(enumerate(dataloader), total=len(dataloader))  # Progress bar
        # for Every Image and Mask as data and targets over Batch ID
        for batch_idx, (data, targets) in loop:
            # Training Mode
            if batch_idx +5 < len(dataloader):
                model.train()
                # Squeeze or Remove batch size generated by stacking and Tensor COnversion
                targets = targets.squeeze(1)
                # Add Data and Targets to CUDA
                data, targets = data.to(device), targets.to(device)

                # Forward pass Data to the Model
                outputs = model(data)

                # Confusion Matrix Data for Intersection Over Union Calculation
                # Get Stats method from Segmentation Models returns tp,tn,fp,fn values for calculations over Multi Label (Multi CHannel Masks)
                # Threshold of 0.5 acts as activation.
                tp, fp, fn, tn = get_stats(outputs.int(), targets.int(), mode="multilabel",threshold=0.5)

                # IOU = TP / (TP+FP+FN) then Average over all channels
                iouscore = iou_score(tp, fp, fn, tn, reduction="micro")

                # Loss Calculation through Dice Loss or Jacard Loss
                loss = criterion(outputs, targets)

                # Zero Gradients so they dont accumulate over Iterations
                optimizer.zero_grad()
                # Back Propagating Loss for Model Corrections
                loss.backward()
                # Stepping the Optimizer to Optimize the Model Parameters
                optimizer.step()
                # Set Descriptions
                loop.set_description(f'Epoch {epoch+1}/{epochs}')
                loop.set_postfix(loss=loss.item(),iou_score = iouscore.item())

            else:
                model.eval()
                targets = targets.squeeze(1)
                data, targets = data.to(device), targets.to(device)

                # Forward Pass
                outputs = model(data)
                tp, fp, fn, tn = get_stats(outputs.int(), targets.int(), mode='multilabel',threshold=0.5)
                iouscore = iou_score(tp, fp, fn, tn, reduction="micro")

                # Loss Calculation
                loss = criterion(outputs, targets.long())  # Ensure targets are long type

                print(f" Epoch under Validation {batch_idx}, Loss: {loss.item()}, IOUScore {iouscore.item()}")
                #loop.set_description(f'Epoch {epoch+1}/{epochs}')
                #loop.set_postfix(loss=loss.item(),iou_score = iouscore.item())
        if epoch%10 == 0:
            # Save Checkpoint
            torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                'iou_score': iou_score
                    }, f"checkpoint_{checkpointName}{epoch+1}.pth")

In [ ]:
def visualizeImageandMasks(image,output,target):

    col = 3
    row = 1
    fig, ax = plt.subplots(row,col)
    ax[0].imshow(image)
    ax[0].set_title("Image")
    ax[1].imshow(output)
    ax[1].set_title("Output")
    ax[2].imshow(target)
    ax[2].set_title("Target")
    plt.show()

## Car Parts Segmentation

In [ ]:
CarPartsClasses,CarPartsClasses_ID = retrieve_meta_data("/kaggle/input/car-parts-and-car-damages/Car damages dataset/meta.json")

In [ ]:
transform = transforms.Compose([transforms.Resize(sizes),
    transforms.ToTensor()])


In [ ]:
CarPartsData =  CarDataLoader(transform,carparts_imgs,carparts_anns,CarPartsClasses_ID,sizes)

In [ ]:
image,mask = CarPartsData[0]

In [ ]:
plt.imshow(image.permute(1,2,0))

In [ ]:
mask.shape

In [ ]:
plt.imshow(mask.squeeze(0).argmax(dim=0))

In [ ]:
mask.min(),mask.max()

In [ ]:
len(CarPartsClasses_ID)+1

In [ ]:
dataloader = DataLoader(CarPartsData,batch_size=20,shuffle=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

In [ ]:
device

## Dice Loss

Dice loss, also known as the Sørensen-Dice coefficient or Dice's coefficient, is a statistical tool that measures the similarity between two datasets. It is defined as the ratio of the size of the intersection to the size of the union of two datasets: 2TP / (2TP + FP + FN) where TP is the number of true positives, FP is the number of false positives, and FN is the number of false negatives. Dice loss is a measure of dissimilarity between two datasets, and it ranges from 0 (denoting total dissimilarity) to 1 (denoting perfect similarity).


In [ ]:
learning_rate = 0.0001
epochs = 50

# Model, Loss, and Optimizer
model = MAnet(classes = len(CarPartsClasses_ID)+1,encoder_weights="imagenet")
model.to(device)
criterion = DiceLoss(classes = len(CarPartsClasses_ID)+1,mode="multilabel")

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

TrainandValidate(model,optimizer,criterion,dataloader,learning_rate,epochs,"MAnet",classes = len(CarPartsClasses_ID)+1)


In [ ]:
for filename in os.listdir("/kaggle/working/"):
    if filename.startswith("checkpoint"):
        os.remove(os.path.join("/kaggle/working/", filename))
        print(filename)

In [ ]:
torch.save({
                'epoch': 34,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f"MANet Model.pth")

In [ ]:
for i in range(10):
    x = random.randrange(1,len(CarPartsData))
    output1 = model(CarPartsData[x][0].unsqueeze(0).to('cuda'))
    image_viz = CarPartsData[x][0].cpu()
    output1 = output1.squeeze(0).detach().cpu()
    target = CarPartsData[x][1].squeeze(0).argmax(dim=0)
    full_mask = output1.argmax(dim=0)
    visualizeImageandMasks(image_viz.permute(1,2,0),full_mask,target)
    plt.show()


## Model Results for Car Parts Identification
The IOU (Intersection over Union) score is 0.86, which indicates that the model is able to accurately segment the objects in the images. An IOU score of 1.0 would indicate a perfect segmentation, so 0.86 is a good score.

The loss is 0.16, which is relatively low. Lower loss values indicate that the model is able to predict the correct class labels for the pixels in the images.

The validation loss is 0.20, which is slightly higher than the training loss. This is expected, as the validation set is not seen by the model during training, so it's a good indicator of how well the model will perform on unseen data.

The IOU score on the validation set is 0.86, which is the same as the training set. This indicates that the model is able to generalize well to unseen data.

Overall, the model seems to have performed well on both the training and validation sets.

## Car Damage Detection

In [ ]:
cardamages_imgs = "/kaggle/input/car-parts-and-car-damages/Car parts dataset/File1/img/"
cardamages_anns = "/kaggle/input/car-parts-and-car-damages/Car parts dataset/File1/ann/"
sizes=(320,320)

In [ ]:
CarDamagesClasses,CarDamagesClasses_ID = retrieve_meta_data("/kaggle/input/car-parts-and-car-damages/Car parts dataset/meta.json")

In [ ]:
CarDamagesData =  CarDataLoader(transform,cardamages_imgs,cardamages_anns,CarDamagesClasses_ID,sizes)

In [ ]:
image,mask = CarDamagesData[0]

In [ ]:
plt.imshow(image.squeeze(0).permute(1,2,0))

In [ ]:
plt.imshow(mask.squeeze(0).argmax(dim=0))

In [ ]:
print(len(CarDamagesClasses_ID)+1)

In [ ]:
damagedataloader = DataLoader(CarDamagesData,batch_size=20,shuffle=True)


## Jaccard Loss

Jaccard loss, also known as Intersection over Union (IoU), is another loss function used to evaluate the performance of a semantic segmentation model. It is defined as the size of the intersection divided by the size of the union of two datasets: TP / (TP + FP + FN). Like Dice loss, Jaccard loss is a measure of dissimilarity between two datasets, and it ranges from 0 (denoting total dissimilarity) to 1 (denoting perfect similarity).

In [ ]:
learning_rate = 0.0001
epochs = 30

# Model, Loss, and Optimizer
model_damages = MAnet(classes = len(CarDamagesClasses_ID)+1,encoder_weights="imagenet")
model_damages.to(device)
criterion_damages = JaccardLoss(classes = len(CarDamagesClasses_ID)+1,mode="multilabel")

optimizer_damages = torch.optim.Adam(model_damages.parameters(), lr=learning_rate)

TrainandValidate(model_damages,optimizer_damages,criterion_damages,damagedataloader,learning_rate,epochs,"MAnet_damages",classes = len(CarDamagesClasses_ID)+1)


In [ ]:
for i in range(10):
    x = random.randrange(1,len(CarDamagesData))
    output1 = model_damages(CarDamagesData[x][0].unsqueeze(0).to('cuda'))
    image_viz = CarDamagesData[x][0].cpu()
    output1 = output1.squeeze(0).detach().cpu()
    target = CarDamagesData[x][1].squeeze(0).argmax(dim=0)
    full_mask = output1.argmax(dim=0)
    visualizeImageandMasks(image_viz.permute(1,2,0),full_mask,target)
    plt.show()

In [ ]:
torch.save({
                'epoch': 30,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f"MANet Damages Model.pth")

# Trial 2

With Dice Loss

In [ ]:
learning_rate = 0.0001
epochs = 30

# Model, Loss, and Optimizer
model_damages = MAnet(classes = len(CarDamagesClasses_ID)+1,encoder_weights="imagenet")
model_damages.to(device)
criterion_damages = DiceLoss(classes = len(CarDamagesClasses_ID)+1,mode="multilabel")

optimizer_damages = torch.optim.Adam(model_damages.parameters(), lr=learning_rate)

TrainandValidate(model_damages,optimizer_damages,criterion_damages,damagedataloader,learning_rate,epochs,"MAnet_damages",classes = len(CarDamagesClasses_ID)+1)
